In [1]:
import sys, torch
import pandas as pd 
print(sys.executable)
print(torch.__version__, torch.version.cuda)

[HAMI-core Msg(6108:140666242594624:libvgpu.c:836)]: Initializing.....
[HAMI-core Warn(6108:140666242594624:multiprocess_memory_limit.c:592)]: Kick dead proc 5562
[HAMI-core Warn(6108:140666242594624:multiprocess_memory_limit.c:592)]: Kick dead proc 5658


/usr/bin/python3.12
2.7.1+cu126 12.6


In [2]:
!pip install huggingface-hub

Looking in indexes: http://pip.58dns.org/simple


In [3]:
%%capture #不显示安装过程
!pip install unsloth
# Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [4]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 本地模型目录路径
local_model_path = "/code/ysh/finetuning/data/models/llama-3-8b-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=local_model_path,  # <-- 使用本地路径
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("模型和分词器已从本地路径成功加载。")

[HAMI-core Msg(6108:140666242594624:libvgpu.c:852)]: Initialized


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


[HAMI-core Msg(6206:140219002754880:libvgpu.c:836)]: Initializing.....
[HAMI-core Msg(6206:140219002754880:libvgpu.c:852)]: Initialized


Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
Standard import failed for UnslothGKDTrainer: No module named 'UnslothGKDTrainer'. Using tempfile instead!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.25 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
###从这里开始准备数据集

In [ ]:
pwd

In [ ]:
from datasets import load_dataset
dataset = load_dataset("parquet",data_files='./data/alpaca-gpt4/train.parquet', split = "train")
print(dataset.column_names)

In [ ]:
print(dataset.column_names)

In [ ]:
# df = pd.read_parquet('./data/alpaca-gpt4/train.parquet')
df = pd.read_parquet('data/finetuning_data.parquet')

In [ ]:
df.head()

In [ ]:
df['text'][1]

In [ ]:
##必须只有两列——一个 instruction 列和一个 output 列。

In [ ]:
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # Select more to handle longer conversations
)

In [ ]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

In [ ]:
##prompt模版

In [ ]:
##Alpaca 格式
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
## Llama-3 的提示格式，必须使用 instruct 而不是 base 
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

In [ ]:
##ChatML 格式
chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

In [ ]:
##最终使用openai模版，一问一答格式
chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                    # Change below!
    {"role": "user", "content": "Filter 5 items that the user may be interested in from candidate items based on the user's personal profile.+UserID: 124, UserProfile: UserID:124 | Gender:M | Age:56 | Occupation:7 | Zip:91356 | RatedMovies:902:4;1544:3;1343:4;3101:4;3107:4;2302:4;527:4;1650:5;2858:3;457:3;3418:5;1358:5;3078:5;1693:4;678:5;531:4;2912:4;1213:4 | FavoriteGenres:Action|Adventure|Children's|Comedy|Crime|Drama|Romance|Sci-Fi|Thriller|War, CandidateItems: 1094:Drama|Romance|War;34:Children's|Comedy|Drama;1179:Crime|Drama|Film-Noir;3125:Drama;3298:Drama;503:Drama;1653:Drama|Sci-Fi|Thriller;1520:Romance;930:Film-Noir|Romance|Thriller;2821:Adventure|Drama;3167:Drama;2153:Action|Adventure;904:Mystery|Thriller;1950:Drama|Mystery;1047:Action|Thriller;1342:Horror;1821:Comedy|Romance;1477:Romance;2692:Action|Crime|Romance;3857:Thriller;2010:Sci-Fi;940:Action|Adventure;2167:Action|Adventure|Horror;3351:Horror;1910:Action|Comedy|Crime;734:Comedy;3894:Drama;618:Comedy|Romance;3747:Drama;2008:Crime|Drama|Film-Noir;429:Comedy;735:Comedy|Horror;1947:Musical|Romance;3262:Drama|Mystery;3674:Adventure|Children's;2799:Comedy;1942:Drama;2884:Comedy|Romance;2174:Comedy|Fantasy;672:Drama;1178:Drama|War;581:Documentary;3796:Romance|Thriller;166:Comedy|Drama;3357:Drama|Romance;2130:Crime|Drama|Romance;2072:Comedy;333:Comedy;424:Drama;690:Romance"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "Filter 5 items that the user may be interested in from candidate items based on the user's personal profile.+UserID: 124, UserProfile: UserID:124 | Gender:M | Age:56 | Occupation:7 | Zip:91356 | RatedMovies:902:4;1544:3;1343:4;3101:4;3107:4;2302:4;527:4;1650:5;2858:3;457:3;3418:5;1358:5;3078:5;1693:4;678:5;531:4;2912:4;1213:4 | FavoriteGenres:Action|Adventure|Children's|Comedy|Crime|Drama|Romance|Sci-Fi|Thriller|War, CandidateItems: 1094:Drama|Romance|War;34:Children's|Comedy|Drama;1179:Crime|Drama|Film-Noir;3125:Drama;3298:Drama;503:Drama;1653:Drama|Sci-Fi|Thriller;1520:Romance;930:Film-Noir|Romance|Thriller;2821:Adventure|Drama;3167:Drama;2153:Action|Adventure;904:Mystery|Thriller;1950:Drama|Mystery;1047:Action|Thriller;1342:Horror;1821:Comedy|Romance;1477:Romance;2692:Action|Crime|Romance;3857:Thriller;2010:Sci-Fi;940:Action|Adventure;2167:Action|Adventure|Horror;3351:Horror;1910:Action|Comedy|Crime;734:Comedy;3894:Drama;618:Comedy|Romance;3747:Drama;2008:Crime|Drama|Film-Noir;429:Comedy;735:Comedy|Horror;1947:Musical|Romance;3262:Drama|Mystery;3674:Adventure|Children's;2799:Comedy;1942:Drama;2884:Comedy|Romance;2174:Comedy|Fantasy;672:Drama;1178:Drama|War;581:Documentary;3796:Romance|Thriller;166:Comedy|Drama;3357:Drama|Romance;2130:Crime|Drama|Romance;2072:Comedy;333:Comedy;424:Drama;690:Romance"},
    {"role": "assistant", "content": "interested items: 1094:Drama|Romance|War;34:Children's|Comedy|Drama;1179:Crime|Drama|Film-Noir;3125:Drama;3298:Drama"},
    {"role": "user",      "content": "Filter 5 items that the user may be interested in from candidate items based on the user's personal profile.+UserID: 433, UserProfile: UserID:433 | Gender:M | Age:50 | Occupation:6 | Zip:55115 | RatedMovies:527:4;1096:5;21:4;3072:3;2736:3;3421:4;2858:4;2289:3;1148:4;1294:4;3543:5;1394:5;1136:3;1230:5;1304:5;1276:5;3210:5;1288:5;3751:4;1965:4;1223:4;377:3;3614:3;471:3;1393:3;25:2;708:3;361:5;356:4;788:2;11:3;1777:3;339:5;539:5;1569:4;2671:1 | FavoriteGenres:Action|Animation|Children's|Comedy|Drama|Fantasy|Musical|Romance|Sci-Fi|Thriller|War|Western, CandidateItems: 804:Comedy|Romance;3004:Comedy|Romance;3:Comedy|Romance;1353:Comedy|Romance;3705:Action|Adventure|Romance|Thriller;2652:Horror;1572:Drama;3190:Adventure|Sci-Fi;2794:Comedy;1231:Drama;2166:Drama|Romance;793:Drama;3540:Romance|Thriller;2581:Comedy|Romance;998:Action|Crime;544:Action;2593:Comedy;310:Comedy;2428:Horror|Sci-Fi;1029:Animation|Children's|Musical;1981:Horror;1228:Drama;1381:Comedy|Musical|Romance;2204:Thriller;352:Comedy;2081:Animation|Children's|Comedy|Musical|Romance;3615:Animation|Children's;2100:Comedy|Fantasy|Romance;1644:Horror|Mystery|Thriller;417:Comedy|Romance;1255:Comedy|Horror;704:Action|Adventure;494:Action|Thriller;658:Drama;1345:Horror;1824:Comedy|Thriller;1784:Comedy|Drama;1322:Horror;1845:Comedy|Thriller;1797:Documentary;2862:Drama;3689:Comedy;3358:Comedy|Romance;3919:Horror;853:Drama;2815:Action|War;415:Comedy|Thriller;2921:Western;5:Comedy;1221:Action|Crime|Drama"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

In [ ]:
pwd

In [ ]:
##保存、加载微调模型
# 这仅保存了 LoRA 适配器，并未保存整个模型。要保存为 16 位或 GGUF，请用下面的
model.save_pretrained("outputs/lora_model") # Local saving
tokenizer.save_pretrained("outputs/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
##如果想加载刚刚保存的 LoRA 适配器以进行推理，请将 False 更改为 True
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "outputs/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
pass

messages = [                    # Change below!
    {"role": "user", "content": "Filter 5 items that the user may be interested in from candidate items based on the user's personal profile.+UserID: 433, UserProfile: UserID:433 | Gender:M | Age:50 | Occupation:6 | Zip:55115 | RatedMovies:527:4;1096:5;21:4;3072:3;2736:3;3421:4;2858:4;2289:3;1148:4;1294:4;3543:5;1394:5;1136:3;1230:5;1304:5;1276:5;3210:5;1288:5;3751:4;1965:4;1223:4;377:3;3614:3;471:3;1393:3;25:2;708:3;361:5;356:4;788:2;11:3;1777:3;339:5;539:5;1569:4;2671:1 | FavoriteGenres:Action|Animation|Children's|Comedy|Drama|Fantasy|Musical|Romance|Sci-Fi|Thriller|War|Western, CandidateItems: 804:Comedy|Romance;3004:Comedy|Romance;3:Comedy|Romance;1353:Comedy|Romance;3705:Action|Adventure|Romance|Thriller;2652:Horror;1572:Drama;3190:Adventure|Sci-Fi;2794:Comedy;1231:Drama;2166:Drama|Romance;793:Drama;3540:Romance|Thriller;2581:Comedy|Romance;998:Action|Crime;544:Action;2593:Comedy;310:Comedy;2428:Horror|Sci-Fi;1029:Animation|Children's|Musical;1981:Horror;1228:Drama;1381:Comedy|Musical|Romance;2204:Thriller;352:Comedy;2081:Animation|Children's|Comedy|Musical|Romance;3615:Animation|Children's;2100:Comedy|Fantasy|Romance;1644:Horror|Mystery|Thriller;417:Comedy|Romance;1255:Comedy|Horror;704:Action|Adventure;494:Action|Thriller;658:Drama;1345:Horror;1824:Comedy|Thriller;1784:Comedy|Drama;1322:Horror;1845:Comedy|Thriller;1797:Documentary;2862:Drama;3689:Comedy;3358:Comedy|Romance;3919:Horror;853:Drama;2815:Action|War;415:Comedy|Thriller;2921:Western;5:Comedy;1221:Action|Crime|Drama"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)